In [ ]:
############### Converting the oneHot encoded data into a sparse matrix, to vastly reduce size

# The problem is that the pickle file is too big, because the onehot encoded dataset is huge. 
# So I find columns which have string values but that can't be onehot encoded because they have too many unique values.
# 3 of these are dates, the other 5 all seem to be some kind of ID.
# These strings are preventing the dataframe from being converted to a sparse matrix, which would be much smaller, and would be hard to use and probably not very useful in ML models
# I convert the dates into datetime objects, and thence to timestamps, which are floats. The other 5 columns I drop.

# Because the ohc.pkl generated by the previous code is too large to store on drive, I am running this on my local machine. This code won't run on colab
# The ohc1 generated by this code, on the other hand, is one tenth the siye and it can be used on colab


import pickle
import pandas as pd
import numpy as np
import scipy



#The problem is that the pickle file is too big, because the onehot encoded dataset is huge

x_path = "C:\\Users\\krish\\ohc.pkl" 
y_path = "C:\\Users\\krish\\depVar"

# Then you can load it onto python and unpickle it, into a pandas dataframe
with open(x_path, 'rb') as ifile:
  Xd =pickle.load(ifile)
with open(y_path, 'rb') as ifile:
  Yd =pickle.load(ifile)

In [ ]:
Xd.shape

(735849, 3064)

In [ ]:
Yd.shape

(735849,)

In [ ]:
X = scipy.sparse.csr_matrix(Xd.values)

AttributeError: module 'scipy' has no attribute 'sparse'

In [ ]:
sparse_Xd = Xd.astype(pd.SparseDtype("float64",0))

In [ ]:
Xcsr = sparse_Xd.sparse.to_coo().tocsr()

In [ ]:
#Xcsr.shape

with open("ohc1", "wb") as ofile:
    pickle.dump(Xcsr, ofile)


In [ ]:
i = 0
for tp in Xd.dtypes:
    if tp != "float64":
        print(tp)

|S7
|S18
|S18
|S9
|S9
|S18
|S9
|S167


In [ ]:
#dropCol2 = []
for col in Xd.columns:
    if Xd[col].dtype != "float64":
        #dropCol2.append(col)
        print(col)

In [ ]:
Xd["m_ids_owner"]

0         b'58134383397900000'
1         b'58134383397900000'
2         b'58134383397900000'
3         b'58134383397900000'
4         b'58134383397900000'
                  ...         
735844                  b'nan'
735845                  b'nan'
735846                  b'nan'
735847                  b'nan'
735848                  b'nan'
Name: m_ids_owner, Length: 735849, dtype: bytes144

In [ ]:
from datetime import datetime

In [ ]:
dat = datetime.strptime(Xd["trx_date"][5].decode('UTF-8'), "%d%b%Y")

In [ ]:
dat

datetime.datetime(2013, 10, 2, 0, 0)

In [ ]:
type(dat.timestamp())

float

In [ ]:
DC = ["first_trx_date_acc", "last_trx_date_acc", "trx_date"]
first_trx_date_acc_f = []
last_trx_date_acc_f = []
trx_date_f = []

i = 0
for dc in DC:
    
    L = []
        
    for d in Xd[dc]:
        try:
            dat = datetime.strptime(d.decode('UTF-8'), "%d%b%Y")
            f = dat.timestamp()
            L.append(f)
        except:
            L.append(np.nan)
    
    if i == 0:
        first_trx_date_acc_f = L
    if i == 1:
        last_trx_date_acc_f = L
    if i == 2:
        trx_date_f = L
        
    i += 1

In [ ]:
Xd['first_trx_date_acc_f'] = first_trx_date_acc_f

In [ ]:
Xd["last_trx_date_acc_f"] = last_trx_date_acc_f
Xd["trx_date_f"] = trx_date_f

In [ ]:
Xd.shape

(735849, 3059)

In [ ]:
Xd = Xd.drop(dropCol2, axis = 1)